In [40]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import cv2 as cv
import numpy as np

In [41]:
img = cv.imread("C:/Users/dev/Desktop/LiveHDRI/OpenCV_test/imageset/landscape.jpg")

gray_img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

gray_img = gray_img.astype(np.float32)
print(gray_img.shape)
print(gray_img.dtype)

(551, 980)
float32


In [42]:
#testimg = gray_img.astype(np.uint8)
#cv.imshow("landscape",testimg)
#cv.waitKey(0)

In [43]:
noise = np.random.randn(gray_img.shape[0],gray_img.shape[1])
noise = noise.astype(np.float32)
print(noise[1,1].dtype)
print(noise.shape)

float32
(551, 980)


In [44]:
result_img = np.zeros((gray_img.shape[0],gray_img.shape[1]),dtype='float32')
print(result_img.shape)

(551, 980)


In [45]:
print(gray_img.nbytes)

2159920


In [46]:
img_gpu = cuda.mem_alloc(gray_img.nbytes)
cuda.memcpy_htod(img_gpu,gray_img)

img_noise_gpu = cuda.mem_alloc(noise.nbytes)
cuda.memcpy_htod(img_noise_gpu,noise)

img_result_gpu = cuda.mem_alloc(result_img.nbytes)
cuda.memcpy_htod(img_result_gpu,result_img)


LogicError: cuMemAlloc failed: an illegal memory access was encountered

In [ ]:
kernel = SourceModule("""
    __global__ void addnoise(float* input, float* output, float* noise){
        //int blockId = (gridDim.x * blockIdx.y) + blockIdx.x;
        //int threadId = (blockId * (blockDim.x * blockDim.y)) + (threadIdx.y * blockDim.x) + threadIdx.x;

        int block_id =
            blockIdx.x +
            blockIdx.y * gridDim.x;

        int block_offset =
            block_id *
            blockDim.x * blockDim.y * blockDim.z;

        int thread_offset =
            threadIdx.x +
            threadIdx.y * blockDim.x;

        int id = block_offset + thread_offset;

        //int xIndex = blockIdx.x * blockDim.x + threadIdx.x;
        //int yIndex = blockIdx.y * blockDim.y + threadIdx.y;
        //if ( xIndex < width && yIndex < height) {
        //    output[xIndex][yIndex] = input[xIndex][yIndex]; //+ 10*noise[idx];
        //}

        output[id] = input[id] + 10*noise[id];
    }
""")

In [ ]:
bdim = (32,32)
dx, mx = divmod(gray_img.shape[0], bdim[0])
dy, my = divmod(gray_img.shape[1], bdim[1])

gdim = ((dx + (mx>0)) *bdim[0], (dy + (my>0)) * bdim[1])

print(bdim)
print(gdim)
func = kernel.get_function("addnoise")
func(img_gpu,img_result_gpu,img_noise_gpu, block=bdim, grid=gdim)

In [ ]:
cuda.memcpy_dtoh(result_img,img_result_gpu)

In [ ]:
result_img = result_img.astype(np.uint8)
print(result_img.shape)
print(result_img)
cv.imshow("gpu_result",result_img)
cv.waitKey(0)